In [3]:
# Cell 1: Setup and imports
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.noise import ReadoutError
import seaborn as sns

# Create a simple test circuit (based on your VQE work)
def create_test_circuit(params):
    """Create test circuit similar to VQE ansatz"""
    qc = QuantumCircuit(4)
    
    # Initial state preparation
    qc.x(0)
    qc.x(1)
    
    # Parameterized gates (like VQE)
    qc.ry(params[0], 0)
    qc.ry(params[1], 1)
    qc.cx(0, 2)
    qc.cx(1, 3)
    qc.ry(params[2], 2)
    qc.ry(params[3], 3)
    
    # Add measurements
    qc.measure_all()
    
    return qc

# Test parameters
test_params = [0.5, 1.0, 0.3, 0.8]
test_circuit = create_test_circuit(test_params)

print("Quantum Error Analysis Project Setup")
print("Test circuit created:")
print(test_circuit.draw())
print(f"Circuit depth: {test_circuit.depth()}")
print(f"Number of qubits: {test_circuit.num_qubits}")

Quantum Error Analysis Project Setup
Test circuit created:
        ┌───┐┌─────────┐                      ░ ┌─┐         
   q_0: ┤ X ├┤ Ry(0.5) ├──■───────────────────░─┤M├─────────
        ├───┤└┬───────┬┘  │                   ░ └╥┘┌─┐      
   q_1: ┤ X ├─┤ Ry(1) ├───┼────■──────────────░──╫─┤M├──────
        └───┘ └───────┘ ┌─┴─┐  │  ┌─────────┐ ░  ║ └╥┘┌─┐   
   q_2: ────────────────┤ X ├──┼──┤ Ry(0.3) ├─░──╫──╫─┤M├───
                        └───┘┌─┴─┐├─────────┤ ░  ║  ║ └╥┘┌─┐
   q_3: ─────────────────────┤ X ├┤ Ry(0.8) ├─░──╫──╫──╫─┤M├
                             └───┘└─────────┘ ░  ║  ║  ║ └╥┘
meas: 4/═════════════════════════════════════════╩══╩══╩══╩═
                                                 0  1  2  3 
Circuit depth: 5
Number of qubits: 4


In [4]:
# Cell 2: Create realistic noise models (fixed version)
def create_noise_models():
    """Create different noise models representing real quantum hardware"""
    
    noise_models = {}
    
    # 1. Low noise model (optimistic)
    low_prob = 0.005  # 0.5% error rate
    low_error_1q = depolarizing_error(low_prob, 1)
    low_error_2q = depolarizing_error(low_prob * 2, 2)
    
    noise_low = NoiseModel()
    noise_low.add_all_qubit_quantum_error(low_error_1q, ['ry', 'x'])
    noise_low.add_all_qubit_quantum_error(low_error_2q, ['cx'])
    
    noise_models['Low_Noise'] = noise_low
    
    # 2. Medium noise model (realistic)
    med_prob = 0.01  # 1% error rate
    med_error_1q = depolarizing_error(med_prob, 1)
    med_error_2q = depolarizing_error(med_prob * 2, 2)
    
    noise_med = NoiseModel()
    noise_med.add_all_qubit_quantum_error(med_error_1q, ['ry', 'x'])
    noise_med.add_all_qubit_quantum_error(med_error_2q, ['cx'])
    
    noise_models['Medium_Noise'] = noise_med
    
    # 3. High noise model (pessimistic)
    high_prob = 0.02  # 2% error rate
    high_error_1q = depolarizing_error(high_prob, 1)
    high_error_2q = depolarizing_error(high_prob * 2, 2)
    
    noise_high = NoiseModel()
    noise_high.add_all_qubit_quantum_error(high_error_1q, ['ry', 'x'])
    noise_high.add_all_qubit_quantum_error(high_error_2q, ['cx'])
    
    noise_models['High_Noise'] = noise_high
    
    # 4. Realistic model with readout errors
    realistic_noise = NoiseModel()
    realistic_noise.add_all_qubit_quantum_error(med_error_1q, ['ry', 'x'])
    realistic_noise.add_all_qubit_quantum_error(med_error_2q, ['cx'])
    
    # Add readout errors (5% error rate)
    readout_error_prob = 0.05
    readout_error = ReadoutError([[1-readout_error_prob, readout_error_prob],
                                 [readout_error_prob, 1-readout_error_prob]])
    
    realistic_noise.add_all_qubit_readout_error(readout_error)
    
    noise_models['Realistic'] = realistic_noise
    
    return noise_models

# Create noise models
noise_models = create_noise_models()

print("Noise models created:")
for name, model in noise_models.items():
    print(f"   {name}: {len(model.noise_instructions)} gate error types")
    if hasattr(model, 'readout_errors') and len(model.readout_errors) > 0:
        print(f"      + readout errors")

print("\nNoise model details:")
print("Low Noise: 0.5% gate errors")
print("Medium Noise: 1.0% gate errors") 
print("High Noise: 2.0% gate errors")
print("Realistic: 1.0% gate errors + 5% readout errors")

Noise models created:
   Low_Noise: 3 gate error types
   Medium_Noise: 3 gate error types
   High_Noise: 3 gate error types
   Realistic: 4 gate error types

Noise model details:
Low Noise: 0.5% gate errors
Medium Noise: 1.0% gate errors
High Noise: 2.0% gate errors
Realistic: 1.0% gate errors + 5% readout errors


In [5]:
# Cell 3: Run noise analysis experiments
def analyze_circuit_under_noise(circuit, noise_models, shots=1000):
    """Analyze how noise affects circuit execution"""
    
    results = {}
    
    # Ideal (noiseless) simulation
    ideal_simulator = AerSimulator()
    ideal_job = ideal_simulator.run(circuit, shots=shots)
    ideal_result = ideal_job.result()
    ideal_counts = ideal_result.get_counts()
    
    results['Ideal'] = ideal_counts
    
    # Noisy simulations
    for noise_name, noise_model in noise_models.items():
        noisy_simulator = AerSimulator(noise_model=noise_model)
        noisy_job = noisy_simulator.run(circuit, shots=shots)
        noisy_result = noisy_job.result()
        noisy_counts = noisy_result.get_counts()
        
        results[noise_name] = noisy_counts
    
    return results

# Run analysis
print("Running noise analysis experiments...")
circuit_results = analyze_circuit_under_noise(test_circuit, noise_models, shots=2000)

print("Experiments complete!")
for exp_name, counts in circuit_results.items():
    total_counts = sum(counts.values())
    most_common = max(counts.items(), key=lambda x: x[1])
    print(f"{exp_name}: {len(counts)} different outcomes")
    print(f"   Most common: {most_common[0]} ({most_common[1]/total_counts:.1%})")

Running noise analysis experiments...
Experiments complete!
Ideal: 14 different outcomes
   Most common: 1111 (61.3%)
Low_Noise: 16 different outcomes
   Most common: 1111 (56.1%)
Medium_Noise: 16 different outcomes
   Most common: 1111 (56.9%)
High_Noise: 16 different outcomes
   Most common: 1111 (54.0%)
Realistic: 16 different outcomes
   Most common: 1111 (46.8%)


In [7]:
# Cell 4: Analyze and visualize noise effects
def calculate_fidelity(ideal_counts, noisy_counts, total_shots):
    """Calculate fidelity between ideal and noisy results"""
    
    # Convert to probability distributions
    ideal_probs = {}
    noisy_probs = {}
    
    # Get all possible outcomes
    all_outcomes = set(list(ideal_counts.keys()) + list(noisy_counts.keys()))
    
    for outcome in all_outcomes:
        ideal_probs[outcome] = ideal_counts.get(outcome, 0) / total_shots
        noisy_probs[outcome] = noisy_counts.get(outcome, 0) / total_shots
    
    # Calculate fidelity (overlap between distributions)
    fidelity = sum(np.sqrt(ideal_probs[outcome] * noisy_probs[outcome]) 
                  for outcome in all_outcomes)**2
    
    return fidelity

def analyze_error_rates(circuit_results):
    """Analyze error rates for different noise models"""
    
    ideal_counts = circuit_results['Ideal']
    total_shots = sum(ideal_counts.values())
    
    analysis_results = {}
    
    for noise_name, noisy_counts in circuit_results.items():
        if noise_name == 'Ideal':
            continue
            
        # Calculate fidelity
        fidelity = calculate_fidelity(ideal_counts, noisy_counts, total_shots)
        
        # Calculate error rate (1 - fidelity)
        error_rate = 1 - fidelity
        
        # Find most probable outcome in ideal case
        ideal_most_probable = max(ideal_counts.items(), key=lambda x: x[1])
        
        # Success rate (getting the most probable ideal outcome)
        success_rate = noisy_counts.get(ideal_most_probable[0], 0) / total_shots
        
        analysis_results[noise_name] = {
            'fidelity': fidelity,
            'error_rate': error_rate,
            'success_rate': success_rate
        }
    
    return analysis_results

# Perform error analysis
error_analysis = analyze_error_rates(circuit_results)

print("\nERROR ANALYSIS RESULTS:")
print("=" * 60)
print(f"{'Noise Model':<15} {'Fidelity':<10} {'Error Rate':<12} {'Success Rate':<12}")
print("-" * 60)

for noise_name, metrics in error_analysis.items():
    print(f"{noise_name:<15} {metrics['fidelity']:<10.3f} {metrics['error_rate']:<12.3f} {metrics['success_rate']:<12.3f}")

print("\nDefinitions:")
print("Fidelity: Overlap between ideal and noisy probability distributions")
print("Error Rate: 1 - Fidelity")
print("Success Rate: Probability of getting most likely ideal outcome")

# Find the impact of readout errors
med_error = error_analysis['Medium_Noise']['error_rate']
realistic_error = error_analysis['Realistic']['error_rate']
readout_impact = realistic_error - med_error

print(f"\nReadout error impact: +{readout_impact:.3f} additional error rate")


ERROR ANALYSIS RESULTS:
Noise Model     Fidelity   Error Rate   Success Rate
------------------------------------------------------------
Low_Noise       0.993      0.007        0.561       
Medium_Noise    0.991      0.009        0.569       
High_Noise      0.984      0.016        0.540       
Realistic       0.941      0.059        0.468       

Definitions:
Fidelity: Overlap between ideal and noisy probability distributions
Error Rate: 1 - Fidelity
Success Rate: Probability of getting most likely ideal outcome

Readout error impact: +0.050 additional error rate
